In [36]:
from utils import *

In [3]:
from prepare_dataset import prepare_dataset
import torch.utils.data as data_utils

data = prepare_dataset()
train_set = data[0]
valid_set = data[1]

train_loader = data_utils.DataLoader(train_set, batch_size=64, pin_memory=True, shuffle=True)
valid_loader = data_utils.DataLoader(valid_set, batch_size=64,)

from torch.utils.data import ConcatDataset

dataloaders = {'train':train_loader,'valid':valid_loader}
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'valid']}
dataset = ConcatDataset([train_set, valid_set])

In [31]:
from sklearn.model_selection import KFold
from model import EfficientNet
import torch
from tqdm import tqdm
from torch.autograd import Variable
device = torch.device("mps")

kfold = KFold(n_splits=5, shuffle=True)
for fold, (train_idx, valid_idx) in enumerate(kfold.split(dataset)):

    model = EfficientNet.from_name('efficientnet-phospho-B-15')
    model = model.to(device)
   # Define data loaders for training and testing data in this fold
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    print(len(train_idx))
    print(len(valid_idx))
    test_subsampler = torch.utils.data.SubsetRandomSampler(valid_idx)
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                dataset,
                batch_size=64, sampler=train_subsampler)
    validloader = torch.utils.data.DataLoader(
                dataset,
                batch_size=64, sampler=test_subsampler)

    for epoch in tqdm(range(20), position=0, leave=True):
        # print('-' * 60)
        # print('Epoch {}/{}'.format(epoch+1, 20))

        train_corrects = 0.0
        train_loss = 0.0
        train_precision, train_recall, train_f1 = 0.0, 0.0, 0.0

        for _, (inputs, labels) in enumerate(tqdm(trainloader, position=1, leave=True)):
                model.train(True)
                inputs = Variable(inputs.to(device, dtype=torch.float), requires_grad=True)
                labels = Variable(labels.to(device))
                pred = model(inputs) # forward

efficientnet-phospho-B-15
13730
3433


  0%|          | 0/20 [00:00<?, ?it/s]


RuntimeError: Given groups=32, weight of size [32, 1, 3, 3], expected input[64, 8, 134, 17] to have 32 channels, but got 8 channels instead

In [33]:
inputs.shape

torch.Size([64, 1, 263, 15])

In [37]:
import re
import collections
import torch
from torch import nn
from utils import *
from torch.nn import functional as F

BlockArgs = collections.namedtuple('BlockArgs', [
    'kernel_size', 'num_repeat', 'input_filters', 'output_filters',
    'expand_ratio', 'id_skip', 'stride', 'se_ratio'])

# Change namedtuple defaults
# GlobalParams.__new__.__defaults__ = (None,) * len(GlobalParams._fields)
BlockArgs.__new__.__defaults__ = (None,) * len(BlockArgs._fields)

In [38]:
def efficientnet(width_coefficient=None, depth_coefficient=None, dropout_rate=0.2,
                 drop_connect_rate=0.2, image_size=None, num_classes=1):
    """ Creates a efficientnet model. """

    blocks_args = [
        'r1_k3_s11_e1_i32_o16_se0.25',
        'r2_k3_s22_e6_i16_o24_se0.25',
        'r1_k5_s22_e6_i24_o40_se0.25',
        # 'r3_k3_s22_e6_i40_o80_se0.25',
        # 'r3_k5_s11_e6_i80_o112_se0.25',
        # 'r4_k5_s22_e6_i112_o192_se0.25',
        # 'r1_k3_s11_e6_i192_o320_se0.25',
        # 'r1_k3_s11_e6_i24_o48_se0.25',
    ]
    blocks_args = BlockDecoder.decode(blocks_args)

    global_params = GlobalParams(
        batch_norm_momentum=0.99,
        batch_norm_epsilon=1e-3,
        dropout_rate=dropout_rate,
        drop_connect_rate=drop_connect_rate,
        # data_format='channels_last',  # removed, this is always true in PyTorch
        num_classes=num_classes,
        width_coefficient=width_coefficient,
        depth_coefficient=depth_coefficient,
        depth_divisor=8,
        min_depth=None,
        image_size=image_size,
    )

    return blocks_args, global_params

blocks_args, global_params = efficientnet(width_coefficient=1.0, depth_coefficient=1.0, dropout_rate=0.7, image_size=[263, 15])

In [28]:
def _decode_block_string(block_string):
    """ Gets a block through a string notation of arguments. """
    assert isinstance(block_string, str)

    ops = block_string.split('_')
    options = {}
    for op in ops:
        splits = re.split(r'(\d.*)', op)
        if len(splits) >= 2:
            key, value = splits[:2]
            options[key] = value

    # Check stride
    # assert (('s' in options and len(options['s']) == 1) or
    #         (len(options['s']) == 2 and options['s'][0] == options['s'][1]))
    assert (len(options['k']) < 4) and (len(options['s']) < 4), 'stride 또는 kernel_size를 조정하세요'
    if len(options['k'])>=2:
        kernel_size = (int(options['k'][:-1]), int(options['k'][-1]))
    else:
        kernel_size = (int(options['k']),int(options['k']))


    if len(options['s'])>=2:
        stride = (int(options['s'][:-1]), int(options['s'][-1]))
    else:
        stride = (int(options['s']), int(options['s']))

    return BlockArgs(
        kernel_size=kernel_size,
        num_repeat=int(options['r']),
        input_filters=int(options['i']),
        output_filters=int(options['o']),
        expand_ratio=int(options['e']),
        id_skip=('noskip' not in block_string),
        se_ratio=float(options['se']) if 'se' in options else None,
        stride=stride)

In [30]:
_decode_block_string('r1_k223_s224_e1_i32_o16_se0.25')

BlockArgs(kernel_size=(22, 3), num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, stride=(22, 4), se_ratio=0.25)

In [32]:
stride = _decode_block_string('r1_k3_s2221_e1_i32_o16_se0.25').stride
print(stride)
len(stride)

# stride = stride if len(stride) == 2 else [stride[0]] * 2

AssertionError: stride 또는 kernel_size를 조정하세요

In [40]:
global_params, blocks_args

(GlobalParams(width_coefficient=1.0, depth_coefficient=1.0, image_size=[263, 15], dropout_rate=0.7, num_classes=1, batch_norm_momentum=0.99, batch_norm_epsilon=0.001, drop_connect_rate=0.2, depth_divisor=8, min_depth=None, include_top=None),
 [BlockArgs(num_repeat=1, kernel_size=3, stride=[1], expand_ratio=1, input_filters=32, output_filters=16, se_ratio=0.25, id_skip=True),
  BlockArgs(num_repeat=2, kernel_size=3, stride=[2], expand_ratio=6, input_filters=16, output_filters=24, se_ratio=0.25, id_skip=True),
  BlockArgs(num_repeat=1, kernel_size=5, stride=[2], expand_ratio=6, input_filters=24, output_filters=40, se_ratio=0.25, id_skip=True)])

In [23]:
import math

class Conv2dStaticSamePadding(nn.Conv2d):
    """ 2D Convolutions like TensorFlow, for a fixed image size"""

    def __init__(self, in_channels, out_channels, kernel_size, image_size=None, **kwargs):
        super().__init__(in_channels, out_channels, kernel_size,  **kwargs)
        self.stride = self.stride if len(self.stride) == 2 else [self.stride[0]] * 2

        # Calculate padding based on image size and save it
        assert image_size is not None
        ih, iw = image_size if type(image_size) == list else [image_size, image_size]
        kh, kw = self.weight.size()[-2:]
        sh, sw = self.stride
        oh, ow = math.ceil(ih / sh), math.ceil(iw / sw)
        pad_h = max((oh - 1) * self.stride[0] + (kh - 1) * self.dilation[0] + 1 - ih, 0)
        pad_w = max((ow - 1) * self.stride[1] + (kw - 1) * self.dilation[1] + 1 - iw, 0)
        if pad_h > 0 or pad_w > 0:
            self.static_padding = nn.ZeroPad2d((pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2))
        else:
            self.static_padding = nn.Identity()


In [46]:
bn_mom = 1 - global_params.batch_norm_momentum
bn_eps = global_params.batch_norm_epsilon
image_size = global_params.image_size
Conv2d = get_same_padding_conv2d(image_size=image_size)
in_channels = 1
out_channels = round_filters(8, global_params)
print(image_size)
print(Conv2d)
print(out_channels)

[263, 15]
functools.partial(<class 'utils.Conv2dStaticSamePadding'>, image_size=[263, 15])
8


In [49]:
_conv_stem = Conv2d(in_channels, out_channels, kernel_size=(15, 1), stride=(2, 1), bias=False)
_bn0 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)
image_size = calculate_output_image_size(image_size, stride=(2, 1))

In [ ]:
def calculate_output_image_size(input_image_size, stride):
    """Calculates the output image size when using Conv2dSamePadding with a stride.
       Necessary for static padding. Thanks to mannatsingh for pointing this out.

    Args:
        input_image_size (int, tuple or list): Size of input image.
        stride (int, tuple or list): Conv2d operation's stride.

    Returns:
        output_image_size: A list [H,W].
    """
    if input_image_size is None:
        return None
    image_height, image_width = get_width_and_height_from_size(input_image_size)
    stride = stride if isinstance(stride, int) else stride[0]
    image_height = int(math.ceil(image_height / stride))
    image_width = int(math.ceil(image_width / stride))
    return [image_height, image_width]

In [57]:
stride = (2, 1)

In [ ]:
assert (len(options['k']) < 4) and (len(options['s']) < 4), 'stride 또는 kernel_size를 조정하세요'
if len(options['k'])>=2:
    kernel_size = (int(options['k'][:-1]), int(options['k'][-1]))
else:
    kernel_size = (int(options['k']),int(options['k']))


if len(options['s'])>=2:
    stride = (int(options['s'][:-1]), int(options['s'][-1]))
else:
    stride = (int(options['s']), int(options['s']))

In [55]:
image_height, image_width = get_width_and_height_from_size([263, 15])
image_height, image_width
stride = stride if not isinstance(stride, int) else stride[0]
image_height = int(math.ceil(image_height / stride))
image_width = int(math.ceil(image_width / stride))

In [58]:
isinstance(stride, int)

False

In [50]:
image_size

[66, 4]

In [44]:
out_channels

8

In [50]:
bn_mom = 1 - global_params.batch_norm_momentum
bn_eps = global_params.batch_norm_epsilon

# Get static or dynamic convolution depending on image size
image_size = global_params.image_size
Conv2d = get_same_padding_conv2d(image_size=image_size)

In [49]:
_conv_stem = Conv2d(in_channels, out_channels, kernel_size=(15, 1), stride=(2, 1),bias=False)
_conv_stem

Conv2dStaticSamePadding(
  1, 8, kernel_size=(15, 1), stride=(2, 1), bias=False
  (static_padding): ZeroPad2d((0, 0, 7, 7))
)